# Multiprocessing Data Exchange
- Inter Process Communincation (IPC)
- As the two process resides in the different memory location, using a global vars wont work.
- Alternative provided by python: ``Value, Array``
- [Guidlines and Docs](https://docs.python.org/3/library/multiprocessing.html#programming-guidelines)


**Cons:**
- Context Switching in case of multiprocessing is time consuming.
- GIL allows the processes to run on single core only.
- Not good for multiprocessing.
- May suffer from deadlock, race condition.

**Imports**
- ``Value`` can be used for single object.
- ``Array`` for collection of data items.

In [1]:
from multiprocessing import Process, Value, Array, Lock
import time
import os

## Shared Value

**Shared Value among the processes**
- ``Value(<type>, <initial_value>)``
- `i` => Integer
- Can be accessed using: `<Value>.value`

In [2]:
balance = Value('i', 100)
lock = Lock()

**Increase / Decrease balance**
- Each used by two of the processes.

In [3]:
def increase_balance(balance, amount, lock):
    """Increase balance"""
    with lock:
        local_copy = balance.value # get the global balance value
        local_copy += amount # update amount
        time.sleep(0.2) # heavy computation
        balance.value = local_copy # update value
    

def decrease_balance(balance, amount, lock):
    """Decreasae Balance"""
    with lock:
        local_copy = balance.value # get the global balance value
        local_copy -= amount # update amount
        time.sleep(0.2) # heavy computation
        balance.value = local_copy # update value
    
    

**Process Initialization**

In [4]:
processes = []
for i in range(5):
    p_inc_bal = Process(target=increase_balance, args=(balance, 100, lock))
    p_dec_bal = Process(target=decrease_balance, args=(balance, 50, lock))
    processes.append(p_inc_bal)
    processes.append(p_dec_bal)

**Start a process:**

In [5]:
for p in processes:
    p.start()

**Join Process:**
- Wait / block the main process untill the completion of other processes. 

In [6]:
for p in processes:
    p.join()

In [7]:
print(f"Final Balance = {balance.value}")

Final Balance = 350


## Shared Array

In [8]:
def increase_item_val(sh_array, amount, lock):
    """
    Cannot be accessed this way
    for i in sh_array:
        i += amount
    """
    for i in range(0, len(sh_array)):
        with lock:
            time.sleep(0.1)
            sh_array[i] += amount

def decrease_item_val(sh_array, amount, lock):
    for i in range(0, len(sh_array)):
        with lock:
            time.sleep(0.1)
            sh_array[i] -= amount

In [9]:
shared_array = Array('i', [i for i in range(0, 10)])
print(f"Initial Value: {shared_array[:]}")

Initial Value: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [10]:
lock = Lock()

In [11]:
p_inc = Process(target=increase_item_val, args=(shared_array, 10, lock))
p_dec = Process(target=decrease_item_val, args=(shared_array, 5, lock))


In [12]:
p_inc.start()
p_dec.start()

In [13]:
p_inc.join()
p_dec.join()

In [14]:
print(f"Final value: {shared_array[:]}")

Final value: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
